In [ ]:
#these were run before 24.6.25 and reflect interpro data entries from this time

In [ ]:

import Bio
from Bio import SeqIO
from io import StringIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import sys, errno, re, json, ssl
from urllib import request
from urllib.error import HTTPError
from time import sleep
import os
import re
import csv
import pandas as pd
from Bio import Entrez

In [ ]:
#the below function pulls a list of strings that contain 2 lines;
#>fasta metadata as the description
#the amino acid sequence of the protein in question
#it requires a URL that will be shared for all these functions




In [ ]:
### DONT RUN unless your making the pkl file that has the taxonomy lookup table. 
#I made the look up table to try and make the metadata acquisition faster by pre downloading taxid dumps from ncbi
#it only helps sometimes so its a minor speed boost. many times I still have to pull the taxid from the api 

#how the taxonomy look up table was constructed:

taxfilepath = "/PATH" 

# Load nodes.dmp specifically for how the ncbi dmo looks
nodes_cols = ['tax_id', 'parent_tax_id', 'rank', 'embl_code', 'division_id', 'inherited_div_flag',
              'genetic_code_id', 'inherited_GC_flag', 'mitochondrial_genetic_code_id', 'inherited_MGC_flag',
              'GenBank_hidden_flag', 'hidden_subtree_root_flag', 'comments','plastid genetic code id','inherited PGC flag',
              'specified_species',' hydrogenosome genetic code id',' inherited HGC flag ']
nodes = pd.read_csv(f'{taxfilepath}/nodes.dmp', sep='\t\|\t', engine='python', names=nodes_cols, usecols=['tax_id', 'parent_tax_id', 'rank'], dtype=str)

nodes['tax_id'] = nodes['tax_id'].str.strip()
nodes['parent_tax_id'] = nodes['parent_tax_id'].str.strip()
nodes['rank'] = nodes['rank'].str.strip()
print(nodes)
# Load names.dmp with the same considerations
names_cols = ['tax_id', 'name_txt', 'unique_name', 'name_class']
names = pd.read_csv(f'{taxfilepath}/names.dmp', sep='\t\|\t', engine='python', names=names_cols, usecols=['tax_id', 'name_txt', 'name_class'], dtype=str)
names['tax_id'] = names['tax_id'].str.strip()
names['name_txt'] = names['name_txt'].str.strip()
names['name_class'] = names['name_class'].str.strip()
print(names)

scientific_names = names[names['name_class'] == 'scientific name\t|']

# Merge dataframes
taxonomy = pd.merge(nodes, scientific_names, on='tax_id')

#testing the query
print(taxonomy)
print(taxonomy[taxonomy['tax_id'] == '9606'])  # Example using Human tax_id
print(taxonomy[taxonomy['tax_id'] == '9606']['rank'])

taxonomy.to_pickle("/PATH/ncbi_2025_taxonomy_table.pkl")


In [ ]:
import json
import ssl
from urllib import request, error
from time import sleep
import sys
# this is basically all from the intepro API code so you can look there as well
def output_fasta_list(base_url):
    context = ssl._create_unverified_context()
    next_url = base_url
    FASTA_STRINGS = []  

    while next_url:
        HEADER_SEPARATOR = "|"
        try:
            req = request.Request(next_url, headers={"Accept": "application/json"})
            with request.urlopen(req, context=context) as res:
                if res.status == 408:  
                    sleep(61)
                    continue
                elif res.status == 204:  
                    break
                payload = json.loads(res.read().decode())
                next_url = payload.get("next")  
            for item in payload.get("results", []):
                entries = item.get("entry_subset") or item.get("entries", [])
                taxa = item.get("taxa", [{"lineage": []}])
                taxa_header = "|".join(taxa[0].get("lineage", []))

                if entries:
                    fasta_header = (
                        ">" + item["metadata"].get("accession", "unknown") + HEADER_SEPARATOR +
                        entries[0].get("accession", "unknown") + HEADER_SEPARATOR +
                        item["metadata"].get("name", "unknown") + HEADER_SEPARATOR +
                        item["metadata"]["source_organism"].get("taxId", "unknown") + HEADER_SEPARATOR +
                        item["metadata"]["source_organism"].get("scientificName", "unknown") + HEADER_SEPARATOR +
                        taxa_header
                    )
                else:
                    fasta_header = ">" + item["metadata"].get("accession", "unknown") + HEADER_SEPARATOR + item["metadata"].get("name", "unknown")

                fasta_seq = item.get("extra_fields", {}).get("sequence", "")
                fasta_string = fasta_header + "\n" + fasta_seq
                FASTA_STRINGS.append(fasta_string)

        except error.HTTPError as e:
            sys.stderr.write("Error processing URL: " + str(next_url) + "; Error: " + str(e) + "\n")
            if e.code == 408 and attempts < 3:
                attempts += 1
                sleep(61)
                continue
            else:
                break

        sleep(1) 

    return FASTA_STRINGS 


In [ ]:
#This function exports the individual fasta files to a given path
#if you are using windows you must changes the \ in the path to /
#sorry about that, but thats windows baby

def fasta_file_write(output_fasta_list,path):
    fasta_list = output_fasta_list
    pathvar = path
    
    for fasta in fasta_list:
        fasta_string = fasta
        fasta_string_2 = fasta
        fasta_string_name , fasta_string_seq = fasta_string.strip('>').split('\n')
        print(fasta_string_name,fasta_string_seq,'\n')
        filename_list = fasta_string_2.strip('>').split('|')
        filename = filename_list[0]
        rec = SeqRecord(
            Seq(fasta_string_seq),
            id='',
            name='',
            description = fasta_string_name)
        print('SeqRecord object : ',rec,sep='\n')
        records = [rec]
        SeqIO.write(records,'{}/{}.fasta'.format(pathvar,filename),'fasta')
        print('done with : ','{}/{}.fasta'.format(pathvar,filename))

    

In [ ]:
tax_LOT = pd.read_pickle("/PATH/ncbi_2025_taxonomy_table.pkl")

In [ ]:
email = 'bgonzalez2412@gmail.com' 


def fetch_TAXID_data(tax_id, email):
    classification = tax_id
    Entrez.email = email  
    retry_limit = 3  #number of tries i used

    attempts = 0
    while attempts < retry_limit:
        try:
            handle = Entrez.efetch(db="taxonomy", id=str(classification), retmode="xml")
            records = Entrez.read(handle)
            for rec in records:
                rank = rec.get("Rank")
            break  
        except Exception as e:
            print(f"HTTP error: {str(e)} - Sleeping 10s and retrying for ID {classification}")
            sleep(10)
            attempts += 1 
        finally:
            if 'handle' in locals() and handle:
                handle.close()  #
    if attempts == retry_limit:
        print(f"Failed after {retry_limit} retries for ID {classification}")

    return rank



def fetch_LINEAGE_data(lineage_list, email):
    Entrez.email = email  # Always set your email for NCBI's Entrez
    lineage_df = {}
    
    for classification in lineage_list[2:]:  #im skipping the root section of the NCBI taxonome and cellular organisms
        try:
            rank_series = tax_LOT[tax_LOT['tax_id'] == classification]['rank']
            if not rank_series.empty:
                rank = rank_series.iloc[0]  # Use the first item from the series
                if rank:
                    lineage_df[rank] = classification
                    print(f'Rank: {rank}, Tax ID: {classification}')
                else:
                    print(f'No rank available for Tax ID: {classification}')
            else:
                print(f'No entry found for Tax ID: {classification}, fetching from NCBI...')
                rank = fetch_TAXID_data(classification, email)
                if rank and rank != "No rank available":
                    lineage_df[rank] = classification
                else:
                    print(f'Failed to fetch data for Tax ID: {classification} from NCBI.')
        except Exception as e:
            print(f'Error processing Tax ID {classification}: {str(e)}')

    return lineage_df

# testing everything still works:
list_tax = ['1', '0', '2624677', '1644055', '2157', '1236']
email = 'your_email@example.com'  # Replace with your actual email
print(fetch_LINEAGE_data(list_tax, email))


In [ ]:
#this function creates a list of dictionaries. The dictionaries are Key value pairs:
# "accession" : A0X009XJT10 , "IPR" : IPR009908 , etc,....
# I have set it up so that it pulls what I consider to be all the useful information contained in the requested JSON
# dictionary from the interpro pull
# if you want to modify it open the url in the browser to see the actual JSON contents or print them with your own code
# then odify the for loop that contains the metadata section (its around "metadata = {'Accession': accession,.....")

def fetch_data(BASE_URL, email):
    context = ssl._create_unverified_context()
    next_url = BASE_URL
    data_list = []
    bad_urls = []
    
    while next_url:
        tries = 0  # Reset tries for each new URL
        while tries <= 3: # I set limit to 3 worked fine 
            try:
                if next_url in bad_urls:
                    print(f"Skipping known bad URL: {next_url}")
                    next_url = None
                    break 
                    
                req = request.Request(next_url, headers={"Accept": "application/json"})
                res = request.urlopen(req, context=context)
                
                
                tries = 0 
                
                if res.status == 204:
                    print("Received 204. Ending fetch.")
                    next_url = None 
                    break

                payload = json.loads(res.read().decode())
                next_url = payload.get("next", None)

                for item in payload["results"]:
                    entries = None
                    if "entry_subset" in item:
                        entries = item["entry_subset"]
                    elif "entries" in item:
                        entries = item["entries"]
                    
                    if not entries:
                        continue
                    
                    taxa = item['taxa']
                    metadata = {
                        'Accession': item["metadata"]["accession"],
                        'IPR': entries[0]["accession"],
                        'name': item["metadata"]["name"],
                        'gene': item["metadata"]["gene"],
                        'protein_length': entries[0]["protein_length"],
                        'in_alphafold': item["metadata"]["in_alphafold"],
                        'Source_Org_ID': item["metadata"]["source_organism"]["taxId"],
                        'Source_Org_Name': item["metadata"]["source_organism"]["scientificName"],
                        'taxa_range': len(taxa[0]["lineage"]),
                        'Sequence': item["extra_fields"]["sequence"],
                    }

                    lineage_df = fetch_LINEAGE_data(taxa[0]['lineage'], email)
                    metadata.update(lineage_df)
                    data_list.append(metadata)

                sleep(1) # to not pull too fast
                break 

            except HTTPError as e:
                print(f"HTTP Error {e.code} occurred for {next_url}. Try {tries+1} of 3.")
                if e.code == 408:
                    sleep(61)
                elif e.code == 404:
                    sleep(61)
                else:
                    print(f"Unhandled HTTP error {e.code}. Stopping fetch.")
                    next_url = None
                    break 
                
                tries += 1
                if tries > 3:
                    print(f"Failed to fetch URL after {tries} tries: {next_url}. Marking as bad and moving on.")
                    bad_urls.append(next_url)
                    next_url = None 
                    break
            except Exception as e:
                print(f"An unexpected error occurred: {e}")
                next_url = None
                break

    return data_list, bad_urls

In [ ]:
# this simple function just creates a dataframe from the fetch_data list

def create_dataframe(data_list):
    df = pd.DataFrame(data_list)
    return df


def construct_csv(dataframe,pathvar,filename):
    path = pathvar
    name = filename
    working_df = dataframe
    header_df = list(working_df.keys())
    
    working_df.to_csv(path_or_buf="{}/{}.csv".format(path,name),sep=",",header = header_df)
    print("exported : " + name + " to : " + path)
    


In [ ]:
#code used just for fastas
if __name__ == "__main__":
  
  pathvar = '/PATH/IPR012932'

  url =  "https://www.ebi.ac.uk:443/interpro/api/protein/UniProt/entry/InterPro/IPR012932/taxonomy/uniprot/2/?page_size=200&extra_fields=sequence"

# fasta stuff
  email = 'your_email@example.com'  
  fasta_list = output_fasta_list(url)
  print(len(fasta_list))
  fasta_file_write(fasta_list,pathvar)

  print("done VKOR")
    
########################
  pathvar = '/PATH/IPR009908'
   
  url =  "https://www.ebi.ac.uk:443/interpro/api/protein/UniProt/entry/InterPro/IPR009908/taxonomy/uniprot/2/?page_size=200&extra_fields=sequence"

# fasta stuff
  email = 'your_email@example.com'  
  fasta_list = output_fasta_list(url)
  print(len(fasta_list))
  fasta_file_write(fasta_list,pathvar)

  print("done MauE")
    
########################
  pathvar = '/PATH/IPR003752'
   
  url =  "https://www.ebi.ac.uk:443/interpro/api/protein/UniProt/entry/InterPro/IPR003752/taxonomy/uniprot/2/?page_size=200&extra_fields=sequence"

# fasta stuff
  email = 'your_email@example.com'  
  fasta_list = output_fasta_list(url)
  print(len(fasta_list))
  fasta_file_write(fasta_list,pathvar)

  print("done DsbB")

In [ ]:
#IPRs used: 
#    MauE: IPR009908
#    VKOR: IPR012932
#    DsbB: IPR003752
#URLS used for archaea from interpro:
#    VKOR: "https://www.ebi.ac.uk:443/interpro/api/protein/UniProt/entry/InterPro/IPR012932/taxonomy/uniprot/2157/?page_size=200&extra_fields=sequence"
#    mauE: "https://www.ebi.ac.uk:443/interpro/api/protein/UniProt/entry/InterPro/IPR009908/taxonomy/uniprot/2157/?page_size=200&extra_fields=sequence"
#    dsbB: "https://www.ebi.ac.uk:443/interpro/api/protein/UniProt/entry/InterPro/IPR003752/taxonomy/uniprot/2157/?page_size=200&extra_fields=sequence"
#URLS used for bacteria from interpro:
#    VKOR: "https://www.ebi.ac.uk:443/interpro/api/protein/UniProt/entry/InterPro/IPR012932/taxonomy/uniprot/2/?page_size=200&extra_fields=sequence"
#    MauE: "https://www.ebi.ac.uk:443/interpro/api/protein/UniProt/entry/InterPro/IPR009908/taxonomy/uniprot/2/?page_size=200&extra_fields=sequence"
#    DsbB: "https://www.ebi.ac.uk:443/interpro/api/protein/UniProt/entry/InterPro/IPR003752/taxonomy/uniprot/2/?page_size=200&extra_fields=sequence"


In [ ]:
#code used for main data acquisition
if __name__ == "__main__":
  
  pathvar = '/PATH/IPR012932'

  url =  "https://www.ebi.ac.uk:443/interpro/api/protein/UniProt/entry/InterPro/IPR012932/taxonomy/uniprot/2/?page_size=200&extra_fields=sequence"

# fasta stuff
  email = 'your_email@example.com'  
  fasta_list = output_fasta_list(url)
  print(len(fasta_list))
  fasta_file_write(fasta_list,pathvar)
#exporting dataset to csv

  archae_list = fetch_data(url,email)
  archaea_df = create_dataframe(archaea_list)
  name = "bacteria_vkor_set"
  construct_csv(archaea_df,pathvar,name)

  print("done VKOR")
    
########################
  pathvar = '/PATH/IPR009908'
   
  url =  "https://www.ebi.ac.uk:443/interpro/api/protein/UniProt/entry/InterPro/IPR009908/taxonomy/uniprot/2/?page_size=200&extra_fields=sequence"

# fasta stuff
  email = 'your_email@example.com'  
  fasta_list = output_fasta_list(url)
  print(len(fasta_list))
  fasta_file_write(fasta_list,pathvar)
#exporting dataset to csv

  archaea_list = fetch_data(url,email)
  archaea_df = create_dataframe(archaea_list)
  name = "bacteria_maue_set"
  construct_csv(archaea_df,pathvar,name)

  print("done MauE")
    
########################
  pathvar = '/PATH/IPR003752'
   
  url =  "https://www.ebi.ac.uk:443/interpro/api/protein/UniProt/entry/InterPro/IPR003752/taxonomy/uniprot/2/?page_size=200&extra_fields=sequence"

# fasta stuff
  email = 'your_email@example.com'  
  fasta_list = output_fasta_list(url)
  print(len(fasta_list))
  fasta_file_write(fasta_list,pathvar)
#exporting dataset to csv

  archaea_list = fetch_data(url,email)
  archaea_df = create_dataframe(archaea_list)
  name = "bacteria_dsbb_set"
  construct_csv(archaea_df,pathvar,name)

  print("done DsbB")

In [ ]:
#this sesction is for the control set it includes IPR
# for ribsosomal_sub IPR047873: "https://www.ebi.ac.uk:443/interpro/api/protein/UniProt/entry/InterPro/IPR047873/taxonomy/uniprot/2/?page_size=200&extra_fields=sequence"
# for RNA polymeras_sub IPR010243:  "https://www.ebi.ac.uk:443/interpro/api/protein/UniProt/entry/InterPro/IPR010243/taxonomy/uniprot/2/?page_size=200&extra_fields=sequence"
# for signal peptidase IPR036286: "https://www.ebi.ac.uk:443/interpro/api/protein/UniProt/entry/InterPro/IPR036286/taxonomy/uniprot/2/?page_size=200&extra_fields=sequence"

# signal peptidase has far more entries, well see how that filters down into species counts

In [ ]:
#code used for main data acquisition
if __name__ == "__main__":
  
  pathvar = '/PATH/IPR047873'

  url =  "https://www.ebi.ac.uk:443/interpro/api/protein/UniProt/entry/InterPro/IPR047873/taxonomy/uniprot/2/?page_size=200&extra_fields=sequence"


#exporting dataset to csv
  email = 'your_email@example.com'   
  bacteria_list = fetch_data(url,email)
  bacteria_df = create_dataframe(bacteria_list)
  name = "bacteria_RIBOSOME_set"
  construct_csv(bacteria_df,pathvar,name)

  print("done Ribosomal subunit")
    
########################
  pathvar = '/PATH/IPR010243'
   
  url =  "https://www.ebi.ac.uk:443/interpro/api/protein/UniProt/entry/InterPro/IPR010243/taxonomy/uniprot/2/?page_size=200&extra_fields=sequence"


#exporting dataset to csv

  bacteria_list = fetch_data(url,email)
  bacteria_df = create_dataframe(bacteria_list)
  name = "bacteria_RIBOSOME_set"
  construct_csv(bacteria_df,pathvar,name)

  print("done RNA poly subunit")
    
########################
  pathvar = 'C:/Users/bgonzale24/Desktop/MauE/notebooks/bacteria_set/IPR036286'
   
  url =  "https://www.ebi.ac.uk:443/interpro/api/protein/UniProt/entry/InterPro/IPR036286/taxonomy/uniprot/2/?page_size=200&extra_fields=sequence"


#exporting dataset to csv
  
  bacteria_list = fetch_data(url,email)
  bacteria_df = create_dataframe(bacteria_list)
  name = "bacteria_RIBOSOME_set"
  construct_csv(bacteria_df,pathvar,name)

  print("done singnal peptidase")

In [ ]:
#This section is for MauG to determine its overlap with MauE
#code used for main data acquisition
#https://www.ebi.ac.uk:443/interpro/api/protein/UniProt/entry/InterPro/IPR026259/taxonomy/uniprot/2/?page_size=200&extra_fields=sequence

if __name__ == "__main__":
  
  pathvar = '/PATH/IPR026259'

  url =  "https://www.ebi.ac.uk:443/interpro/api/protein/UniProt/entry/InterPro/IPR026259/taxonomy/uniprot/2/?page_size=200&extra_fields=sequence"

  email = 'your_email@example.com' 
#exporting dataset to csv

  bacteria_list = fetch_data(url,email)
  bacteria_df = create_dataframe(bacteria_list)
  name = "bacteria_MAUG_set"
  construct_csv(bacteria_df,pathvar,name)

  print("done with MauG")

In [ ]:
#this section is for getting the Mau A data. this is the madh light chain. there are only 1199 proteins
#https://www.ebi.ac.uk:443/interpro/api/protein/UniProt/entry/InterPro/IPR036560/taxonomy/uniprot/2/?page_size=200&extra_fields=sequence

In [ ]:
if __name__ == "__main__":
  
  pathvar = '/PATH/IPR036560'

  url =  "https://www.ebi.ac.uk:443/interpro/api/protein/UniProt/entry/InterPro/IPR036560/taxonomy/uniprot/2/?page_size=200&extra_fields=sequence"
  email = 'your_email@example.com' 

#exporting dataset to csv

  bacteria_list = fetch_data(url,email)
  bacteria_df = create_dataframe(bacteria_list)
  name = "bacteria_MAUA_set"
  construct_csv(bacteria_df,pathvar,name)

  print("done with MauA")